# Exemple Pipeline

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

## Obtenir les titres que nous voulons.

### Les Q500US et Q1500US

Ces groupes d'actions négociables sont appelés "univers", car toutes vos transactions utiliseront ces actions comme leur "univers" d'actions disponibles, ils ne négocieront rien en dehors de ces groupes.

In [2]:
from quantopian.pipeline.filters import Q1500US

Le Q500US et le Q1500US présentent deux avantages principaux. Premièrement, ils réduisent considérablement le risque qu'un ordre ne soit pas exécuté. Deuxièmement, ils permettent des comparaisons plus significatives entre les stratégies car ils seront désormais utilisés comme univers standard pour les algorithmes.

In [3]:
universe = Q1500US()

## Filtrer davantage l'univers avec les classificateurs

Ne prenons que des actions dans le secteur de l'énergie : https://www.quantopian.com/docs/data-reference/morningstar_fundamentals#industry-sector

In [4]:
from quantopian.pipeline.data import morningstar

In [5]:
sector = morningstar.asset_classification.morningstar_sector_code.latest

Alternative:

In [6]:
# from quantopian.pipeline.classifiers.morningstar import Sector
# morningstar_sector = Sector()

In [7]:
energy_sector = sector.eq(309)

## Masques de filtre

Les masques peuvent également être appliqués aux méthodes qui renvoient des filtres comme top, bottom et le percentile_between.

Les masques sont particulièrement utiles lorsque nous voulons appliquer un filtre dans les premières étapes d'un calcul combiné. Par exemple, supposons que nous voulions obtenir les 50 titres dont le prix à l'ouverture est le plus élevé et qui se situent également dans les 10 % supérieurs du volume en dollars. 

Supposons que nous voulions ensuite obtenir le 90ème-100ème percentile de ces titres par le prix de clôture. Nous pouvons le faire avec les éléments suivants :

In [8]:
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

In [9]:
# Facteur de volume en dollar - Dollar volume factor
dollar_volume = AverageDollarVolume(window_length=30)

# Filtre sur les volumes importants en dollars - High dollar volume filter
high_dollar_volume = dollar_volume.percentile_between(90,100)

# Filtre de prix supérieur à l'ouverture (titres à fort volume en dollars)
# Top open price filter (high dollar volume securities)
top_open_price = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)

# Filtre de prix de clôture du premier percentile (volume de dollars élevé, prix d'ouverture des 50 premiers)
# Top percentile close price filter (high dollar volume, top 50 open price)
high_close_price = USEquityPricing.close.latest.percentile_between(90, 100, mask=top_open_price)

## Application des Filtres et des Facteurs

Appliquons nos propres filtres, en suivant certains des exemples ci-dessus. Choisissons les titres suivants :

* Titres de Q1500US
* Les actions qui sont dans le secteur de l'énergie
* Il doit s'agir d'actions fortement négociées sur le marché (en fonction du volume de dollars échangés, elles doivent faire partie des 5% les plus échangés)

Ensuite, nous calculerons la différence en pourcentage comme nous l'avons fait précédemment. En utilisant ce pourcentage de différence, nous créerons une stratégie peu sophistiquée qui court-circuite tout ce qui présente un pourcentage de différence négatif (la différence entre la moyenne sur 10 jours et la moyenne sur 30 jours).

In [10]:
def make_pipeline():
    
    # Univers Q1500US
    base_universe = Q1500US()
    
    # Secteur Energie
    energy_sector = sector.eq(309)
    
    # Masque de l'univers 1500US et du secteur Energie
    base_energy = base_universe & energy_sector
    
    # Volume en dollars (30 jours)
    dollar_volume = AverageDollarVolume(window_length=30)

    # 5% supérieurs en volume en dollars moyens
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    
    # Combiner les filtres
    top_half_base_energy = base_energy & high_dollar_volume
    
    # 10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10, mask=top_half_base_energy)

    # 30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30, mask=top_half_base_energy)

    # Différence en pourcentage
    percent_difference = (mean_10 - mean_30) / mean_30
    
    # Liste des Shorts (position courte ou vente à découvert)
    shorts = percent_difference < 0
    
    # Liste des Longs (position longue)
    longs = percent_difference > 0
    
    # Masque final/filtre pour tout ce qui est en short ou en long
    securities_to_trade = (shorts | longs)
    
    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts,
            'percent_diff':percent_difference
        },
        screen=securities_to_trade
    )

In [11]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

longs  percent_diff shorts
2015-05-05 00:00:00+00:00 Equity(216 [HES])     True      0.036528  False
                          Equity(448 [APA])     True      0.035926  False
                          Equity(455 [APC])     True      0.049153  False
                          Equity(858 [BHI])     True      0.033807  False
                          Equity(1746 [COG])    True      0.058078  False
                          Equity(2368 [DVN])    True      0.046264  False
                          Equity(2564 [EOG])    True      0.032102  False
                          Equity(2621 [VAL])    True      0.060197  False
                          Equity(3443 [HAL])    True      0.049257  False
                          Equity(3647 [HP])     True      0.040991  False
                          Equity(5035 [MRO])    True      0.061598  False
                          Equity(5213 [NBL])    True      0.010443  False
                          Equity(5214 [NBR])    True      0.064133  False
                          Equity(5249 [NE])     True      0.037559  False
                          Equity(5729 [OXY])    True      0.029776  False
                          Equity(6928 [SLB])    True      0.046555  False
                          Equity(7244 [SWN])    True      0.070788  False
                          Equity(7612 [ANDV])   True      0.005997  False
                          Equity(7990 [VLO])   False     -0.017145   True
                          Equity(8214 [WMB])    True      0.018876  False
                          Equity(8347 [XOM])    True      0.017343  False
                          Equity(8461 [CHK])    True      0.014265  False
                          Equity(9038 [RIG])    True      0.048180  False
                          Equity(13176 [CAM])   True      0.082110  False
                          Equity(17436 [PXD])   True      0.010248  False
                          Equity(19249 [RRC])   True      0.087062  False
                          Equity(19336 [WFT])   True      0.049141  False
                          Equity(22784 [FTI])   True      0.054529  False
                          Equity(23112 [CVX])   True      0.018972  False
                          Equity(23998 [COP])   True      0.023902  False
                          Equity(24809 [NOV])   True      0.024940  False
                          Equity(25707 [WLL])   True      0.048205  False
                          Equity(33856 [CLR])   True      0.064304  False
                          Equity(34440 [CXO])   True      0.042184  False
                          Equity(39797 [OAS])   True      0.042388  False
                          Equity(40852 [KMI])   True      0.023016  False
                          Equity(41636 [MPC])   True      0.011952  False
                          Equity(42788 [PSX])   True      0.020911  False

In [12]:
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 38 entries, (2015-05-05 00:00:00+00:00, Equity(216 [HES])) to (2015-05-05 00:00:00+00:00, Equity(42788 [PSX]))
Data columns (total 3 columns):
longs           38 non-null bool
percent_diff    38 non-null float64
shorts          38 non-null bool
dtypes: bool(2), float64(1)
memory usage: 684.0+ bytes


# Exécution de cette stratégie dans l'IDE Quantopian

In [ ]:
from quantopian.algorithm import attach_pipeline,pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume,SimpleMovingAverage
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data import morningstar

def initialize(context):
    
    schedule_function(my_rebalance,date_rules.week_start(),time_rules.market_open(hours=1))
    
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe,'my_pipeline')
    
def my_rebalance(context,data):
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            order_target_percent(security,0)
            
    for security in context.longs:
        if data.can_trade(security):
            order_target_percent(security,context.long_weight)

    for security in context.shorts:
        if data.can_trade(security):
            order_target_percent(security,context.short_weight)




def my_compute_weights(context):
    
    if len(context.longs)==0:
        long_weight = 0
    else:
        long_weight = 0.5 / len(context.longs)
  
    if len(context.shorts)==0:
        short_weight = 0
    else:
        short_weight = -0.5 / len(context.shorts)
    
    return (long_weight,short_weight)






def before_trading_start(context,data):
    context.output = pipeline_output('my_pipeline')
    
    # LONG
    context.longs = context.output[context.output['longs']].index.tolist()
    
    # SHORT
    context.shorts = context.output[context.output['shorts']].index.tolist()


    context.long_weight,context.short_weight = my_compute_weights(context)



def make_pipeline():
    
    # Univers Q1500US
    base_universe = Q1500US()
    
    # Secteur Energie
    sector = morningstar.asset_classification.morningstar_sector_code.latest
    energy_sector = sector.eq(309)
    
    # Masque de l'univers 1500US et du secteur Energie
    base_energy = base_universe & energy_sector
    
    # Volume en dollars (30 jours)
    dollar_volume = AverageDollarVolume(window_length=30)
    
    # 5% supérieurs en volume en dollars moyens
    high_dollar_volume = dollar_volume.percentile_between(95,100)
     
    # Combiner les filtres
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=top_five_base_energy)
    
    # 30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=top_five_base_energy)
    
    # Différence en pourcentage
    percent_difference = (mean_10-mean_30)/mean_30
    
    # Liste des Shorts (position courte ou vente à découvert)
    shorts = percent_difference < 0
    
    # Liste des Longs (position longue)
    longs = percent_difference > 0
    
    # Masque final/filtre pour tout ce qui est en short ou en long
    securities_to_trade = (shorts | longs)
    
    # Retourne le Pipeline
    return Pipeline(columns={
        'longs':longs,
        'shorts':shorts,
        'perc_diff':percent_difference
    },screen=securities_to_trade)